__NOTE:__ For transforming the bboxes by albumentations library make sure you convert the bbox into 'coco' format which is [x_min, y_min, width, height], since it is the most stable. On repeated experimentation it was found that 'yolo' and 'pascal_voc' seems to be broken.

# Importing Library

In [1]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import selectivesearch
import os
from tqdm import tqdm
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import albumentations as A
import json
import timm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hyperparameters

In [2]:
img_aug_p = 0.5
iou_threshold = 0.4
img_crop_size = (224, 224)
img_resize_size = 256

# mean and std-dev taken from imagenet dataset
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

batch_size = 2
num_workers = 8

invTransform = transforms.Compose([transforms.Normalize(mean = [0., 0., 0.], std = [1/std[0], 1/std[1], 1/std[2]]),
                                  transforms.Normalize(mean = [-mean[0], -mean[1], -mean[2]], std = [1., 1., 1.])])

model_name = 'resnet50'
dropout_p = 0.4
num_epochs = 10
learning_rate = 1e-04
num_freeze = 100
freeze = True
lmb = 0.1

save_path = './saved_models/rcnn_checkpoint.pt'

# Helper Functions

In [3]:
# Ref: https://albumentations.ai/docs/examples/example_bboxes/
def draw_bounding_boxes(image, bbox, category):
    x_1, y_1, x_2, y_2 = bbox
    x_1, y_1, x_2, y_2 = int(x_1), int(y_1), int(x_2), int(y_2)
    cv2.rectangle(image, (x_1, y_1), (x_2, y_2), (255, 0, 0), thickness=2, lineType = cv2.LINE_AA)
    ((text_width, text_height), _) = cv2.getTextSize(category, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(image, (x_1, y_1 - int(1.3 * text_height)), (x_1 + text_width, y_1), (0, 0, 0), -1)
    cv2.putText(image, category, (x_1, y_1 - int(0.3 * text_height)), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 255, 255))
    return image

def visualize_bbox(image, bboxes, category_idxs, idx_to_cat):
    img = image.copy()
    for bbox, category_idx in zip(bboxes, category_idxs):
        category = idx_to_cat[category_idx]
        img = draw_bounding_boxes(img, bbox, category)
    plt.figure(figsize = (8,8))
    plt.imshow(img)
    plt.axis('off')

def extract_regions(img, scale = 100):
    _, regions = selectivesearch.selective_search(img, scale = scale, min_size=100)
    candidates = []
    total_area = np.prod(img.shape[:2])
    for region in regions:
        if region['rect'] in candidates:
            continue
        # Remove rectangle which is less than 5% of total image area or greater than image
        elif region['size'] < (0.05*total_area):
            continue
        elif region['size'] > (1*total_area):
            continue
        candidates.append(region['rect'])
    return candidates

def calculate_iou(box_1, box_2, epsilon = 1e-06):
    iou = 0.0
    box_1_area = (box_1[2] - box_1[0]) * (box_1[3] - box_1[1])
    box_2_area = (box_2[2] - box_2[0]) * (box_2[3] - box_2[1])
    x1_inter = max(box_1[0], box_2[0])
    y1_inter = max(box_1[1], box_2[1])
    x2_inter = min(box_1[2], box_2[2])
    y2_inter = min(box_1[3], box_2[3])
    inter_width = (x2_inter - x1_inter)
    inter_height = (y2_inter - y1_inter)
    if (inter_height < 0 or inter_width < 0):
        return 0.0
    inter_area = inter_width*inter_height
    union_area = box_1_area - box_2_area - inter_area
    iou = inter_area/(union_area + epsilon)
    return iou

def save_checkpoint(model, save_path):
    torch.save(model.state_dict(), save_path)
    
def load_checkpoint(model, save_path):
    model.load_state_dict(torch.load(save_path))
    return model

# Loading Data and Visualization

In [4]:
base_folder = pathlib.Path('data/coco')
train_image_folder = base_folder/'train2017'
valid_image_folder = base_folder/'val2017'
train_csv = pd.read_csv(base_folder/'train.csv')
valid_csv = pd.read_csv(base_folder/'valid.csv')
train_csv['path'] = list(map(lambda x:train_image_folder/x, train_csv['file_name']))

# Checking if the x_1 == x_2 or y_1 == y_2 then add +2 for some width or height
train_csv.loc[train_csv['y_1'] == train_csv['y_2'], 'y_2'] = train_csv.loc[train_csv['y_1'] == train_csv['y_2'], 'y_2'] + 2
train_csv.loc[train_csv['x_1'] == train_csv['x_2'], 'x_2'] = train_csv.loc[train_csv['x_1'] == train_csv['x_2'], 'x_2'] + 2

# Transforming width and height column to width and height of the bounding box
train_csv['width'] = train_csv['x_2'].values - train_csv['x_1'].values
train_csv['height'] = train_csv['y_2'].values - train_csv['y_1'].values

# Valid
valid_csv['path'] = list(map(lambda x:valid_image_folder/x, valid_csv['file_name']))

# Checking if the x_1 == x_2 or y_1 == y_2 then add +2 for some width or height
valid_csv.loc[valid_csv['y_1'] == valid_csv['y_2'], 'y_2'] = valid_csv.loc[valid_csv['y_1'] == valid_csv['y_2'], 'y_2'] + 2
valid_csv.loc[valid_csv['x_1'] == valid_csv['x_2'], 'x_2'] = valid_csv.loc[valid_csv['x_1'] == valid_csv['x_2'], 'x_2'] + 2

# Transforming width and height column to width and height of the bounding box
valid_csv['width'] = valid_csv['x_2'].values - valid_csv['x_1'].values
valid_csv['height'] = valid_csv['y_2'].values - valid_csv['y_1'].values
train_csv.head()

,image_id,file_name,width,height,x_1,y_1,x_2,y_2,category_idx,category_name,path
0,9,000000000009.jpg,75,46,376,40,451,86,55,orange,data/coco/train2017/000000000009.jpg
1,9,000000000009.jpg,58,47,465,38,523,85,55,orange,data/coco/train2017/000000000009.jpg
2,9,000000000009.jpg,320,228,311,4,631,232,51,bowl,data/coco/train2017/000000000009.jpg
3,9,000000000009.jpg,316,245,249,229,565,474,56,broccoli,data/coco/train2017/000000000009.jpg
4,9,000000000009.jpg,94,71,364,2,458,73,55,orange,data/coco/train2017/000000000009.jpg


In [5]:
train_labels_json = base_folder/'annotations/instances_train2017.json'
with open(train_labels_json, 'r') as file:
        data = json.load(file)
idx_to_cat = {cat['id']:cat['name'] for cat in data['categories']}
cat_to_idx = {cat['name']:cat['id'] for cat in data['categories']}

# Some of the index are missing from data
idx_not_present = []
min_index = train_csv['category_idx'].min()
max_index = train_csv['category_idx'].max()
for i in range(min_index, max_index+1):
    try:
        _cat_name = idx_to_cat[i]
    except:
         idx_to_cat[i] = 'not_known'

idx_to_cat[0] = 'background'
cat_to_idx['background'] = 0
print(len(idx_to_cat))

In [ ]:
class COCOData(Dataset):
    def __init__(self, df, transforms = None, N = 1000):
        self.df = df
        self.unique_ids = df['image_id'].unique()
        np.random.shuffle(self.unique_ids)
        self.unique_ids = self.unique_ids[:N]
        self.transforms = transforms

    def __len__(self):
        return len(self.unique_ids)
    
    def __getitem__(self, idx):
        image_id = self.unique_ids[idx]
        df_sub = self.df[self.df['image_id'] == image_id]
        # Since all path are same for subset
        file_path = df_sub['path'].iloc[0]
        image = cv2.imread(str(file_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        bboxes = df_sub[['x_1', 'y_1', 'width', 'height']].values
        category_idx = df_sub['category_idx'].values.tolist()
        if self.transforms:
            transformed = self.transforms(image = image, bboxes = bboxes, category_idx=category_idx)
            image = transformed['image']
            bboxes = np.array(transformed['bboxes'])
        H, W = image.shape[:2]
        # If the bounding box is pushed outside the limits so there is no category only background
        try:
            bboxes = np.hstack((bboxes[:, :2], bboxes[:, :2] + bboxes[:,2:]))
        except:
            bboxes = np.array([[0, 0, 0, 0]])
            category_idx = [0]

        # Extract proposed regions and compute pairwise iou with all the bbox present
        candidates = extract_regions(image)
        candidates = np.array([(x, y, x+w, y+h) for x,y,w,h in candidates])
        iou = np.array([[calculate_iou(bb, candidate) for bb in bboxes] for candidate in candidates])
        current_lbls, current_offsets, img_patches = [], [], []

        # Finding offset from the correct bounding boxes based on iou threshold
        for idx, candidate in enumerate(candidates):
            c_x_1, c_y_1, c_x_2, c_y_2 = candidate
            img_patch = image[c_y_1: c_y_2, c_x_1:c_x_2]
            img_patch = cv2.resize(img_patch, (224, 224))
            img_patches.append(img_patch)
            candidate_iou = iou[idx]
            best_iou = np.argmax(candidate_iou)
            iou_value = candidate_iou[best_iou]
            bbox = bboxes[best_iou]
            if iou_value > iou_threshold:
                current_lbls.append(category_idx[best_iou])
            else:
                current_lbls.append(0)

            # Make them in proportion to the image size
            current_offset = np.array([bbox[0] - c_x_1, bbox[1] - c_y_1, bbox[2] - c_x_2, bbox[3] - c_y_2]) / np.array([W, H, W, H])
            current_offsets.append(current_offset)

        # Building Tensor out of Data
        img_patches = np.array(img_patches)
        current_lbls = np.array(current_lbls)
        current_offsets = np.array(current_offsets)
        img_patches = torch.Tensor(img_patches)
        current_lbls = torch.Tensor(current_lbls).long()
        current_offsets = torch.Tensor(current_offsets).float()
        return img_patches, current_lbls, current_offsets
    
    def collate_function(self, batch):
        # Since images may have multiple number of patches, this'll make them all concatenated together
        final_inputs = []
        final_lbls = []
        final_offsets = []
        for current_batch in batch:
            final_inputs.append(current_batch[0])
            final_lbls.append(current_batch[1])
            final_offsets.append(current_batch[2])

        final_inputs = torch.permute(torch.cat(final_inputs), (0, 3, 1, 2))
        final_lbls = torch.cat(final_lbls)
        final_offsets = torch.cat(final_offsets)
        return final_inputs, final_lbls, final_offsets

In [ ]:
train_transforms = A.Compose([A.HorizontalFlip(p = img_aug_p),
                              A.ShiftScaleRotate(p = img_aug_p),
                              A.LongestMaxSize(max_size= img_resize_size),
                              A.PadIfNeeded(img_resize_size, img_resize_size),
                              A.RandomCrop(*img_crop_size),
                              A.Normalize(mean, std, max_pixel_value=255.0),],
                              bbox_params=A.BboxParams('coco', label_fields=['category_idx']))

valid_transforms = A.Compose([A.LongestMaxSize(max_size= img_resize_size),
                              A.PadIfNeeded(img_resize_size, img_resize_size),
                              A.RandomCrop(*img_crop_size),
                              A.Normalize(mean, std, max_pixel_value=255.0),],
                              bbox_params=A.BboxParams('coco', label_fields=['category_idx']))

train_coco = COCOData(train_csv, train_transforms, N = 20000)
valid_coco = COCOData(valid_csv, valid_transforms, N = -1)
train_loader = DataLoader(train_coco, batch_size=batch_size, num_workers = num_workers, drop_last=True, 
                          collate_fn=train_coco.collate_function)
valid_loader = DataLoader(valid_coco, batch_size=batch_size, num_workers=num_workers, 
                         collate_fn=valid_coco.collate_function)

In [ ]:
print(f"No of Train Images: {len(train_coco)}")
print(f"No of Test Images: {len(valid_coco)}")

In [ ]:
temp = next(iter(train_loader))
idx = 7
img = invTransform(temp[0][idx])
img = torch.permute(img, (1, 2, 0))
plt.imshow(img)
plt.title(idx_to_cat[temp[1][idx].item()]);

# Building Model

In [ ]:
class RCNN(nn.Module):
    def __init__(self, base_model = 'resnet50', lmb = 0.1, freeze = True,
                 num_freeze = 100, dropout_p = 0.4, idx_to_cat = idx_to_cat):
        super(RCNN, self).__init__()
        model = timm.create_model(base_model, pretrained=True)
        lin_features = model.num_features
        layers = list(model.children())[:-1]
        layers.extend([nn.Linear(lin_features, 512),
                        nn.ReLU(),
                        nn.BatchNorm1d(512),
                        nn.Dropout(dropout_p),
                        nn.Linear(512, 64),
                        nn.ReLU(),
                        nn.BatchNorm1d(64),
                        nn.Dropout(dropout_p)])
        
        # Backbone architecture
        self.feature_extractor = nn.Sequential(*layers)
        if freeze:
            self.freeze_layers(num_freeze)

        # Classifier
        self.categories = nn.Sequential(nn.Linear(64, len(idx_to_cat)))

        # Offset prediction
        self.offset = nn.Sequential(nn.Linear(64, 4),
                                    nn.Tanh())

        self.feature_extractor.apply(self.initialize_weights)
        self.categories.apply(self.initialize_weights)
        self.offset.apply(self.initialize_weights)

        # Initializing loss and its parameters
        self.lmb = lmb
        self.l1 = nn.L1Loss()
        self.cat_entropy = nn.CrossEntropyLoss()

    def initialize_weights(self, layer):
        if isinstance(layer, nn.Linear):
            torch.nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
            layer.bias.data.fill_(0.0)

    def freeze_layers(self, num_freeze):
        for idx, param in enumerate(self.feature_extractor.parameters()):
            if idx <= num_freeze:
                param.requires_grad = False

    def forward(self, x):
        features = self.feature_extractor(x)
        cat = self.categories(features)
        off = self.offset(features)
        return cat, off
    
    def calculate_loss(self, cat_pred, offset_pred, cat_true, offset_true):
        cat_loss = self.cat_entropy(cat_pred, cat_true)
        idxs = torch.where(cat_true != 0)[0]
        offset_pred = offset_pred[idxs]
        offset_true = offset_true[idxs]
        if len(idxs) > 0:
            regression_loss = self.l1(offset_pred, offset_true)
            final_loss = regression_loss*self.lmb + cat_loss
            return final_loss, cat_loss.detach(), regression_loss.detach()
        else:
            final_loss = cat_loss
            return final_loss, cat_loss.detach(), 0.0

In [ ]:
rcnn = RCNN(base_model=model_name, lmb = lmb, freeze=freeze, num_freeze=num_freeze, dropout_p=dropout_p, idx_to_cat=idx_to_cat)
rcnn = rcnn.to(device)
optimizer = optim.Adam(rcnn.parameters(), lr = learning_rate)
criterion = rcnn.calculate_loss
summary(rcnn, (3, 224, 224));

# Training

In [ ]:
def no_of_correct(imgs, lbls, model):
    imgs, lbls, model = imgs.to(device, non_blocking = True), lbls.to(device, non_blocking = True), model.to(device)
    with torch.no_grad():
        model.eval()
        out = model(imgs)[0]
        _, preds= out.max(-1)
        correct_preds = preds == lbls
        return correct_preds.cpu().numpy().sum(), len(lbls)
    
def train_batch(imgs, lbls, offset_true, model, criterion, optimizer, scaler, valid = False):
    optimizer.zero_grad()
    imgs, lbls, model = imgs.to(device, non_blocking = True), lbls.to(device, non_blocking = True), model.to(device)
    offset_true = offset_true.to(device, non_blocking = True)
    with torch.cuda.amp.autocast():
        out = model(imgs)
        batch_loss, cat_loss, regression_loss = criterion(out[0], out[1], lbls, offset_true)
    if not valid:
        scaler.scale(batch_loss).backward()
        scaler.step(optimizer)
        scaler.update()
    return batch_loss.item(), cat_loss, regression_loss

def train(trainloader, validloader, model, criterion, optimizer, num_epochs, load_check = False,
          best_valid_metric = 0.0, save_path = './saved_models/checkpoint.pt', early_stopping = True, epoch_threshold = 3):    
    # Load From checkpoing
    if load_check:
        model = load_checkpoint(model, save_path)

    scaler = torch.cuda.amp.GradScaler()
    train_accuracies, valid_accuracies = [], []
    train_losses, valid_losses = [], []
    no_change_epoch = 0
    for epoch in range(num_epochs+1):
        batch_loss = 0.0
        model.train()
        pbar = tqdm(trainloader, total = len(trainloader), leave = False)
        for imgs, lbls_true, offset_true in pbar:
            imgs = imgs.to(torch.float32)
            current_batch_loss, current_reg_loss, current_cat_loss = train_batch(imgs, lbls_true, 
                                                                                 offset_true, model, criterion, optimizer, scaler)
            batch_loss += current_batch_loss
            pbar.set_postfix(train_loss = current_batch_loss)

        total_correct = 0
        total = 0
        for imgs, lbls, _ in trainloader:
            imgs = imgs.to(torch.float32)
            batch_correct, batch_total = no_of_correct(imgs, lbls, model)
            total_correct += batch_correct
            total += batch_total

        epoch_train_acc = total_correct/total
        train_accuracies.append(epoch_train_acc)
        epoch_train_loss = batch_loss / len(trainloader)
        train_losses.append(epoch_train_loss)

        # Calculating for Valid data
        batch_loss = 0.0
        total_correct = 0
        total = 0
        for imgs, lbls_true, offset_true in validloader:
            imgs = imgs.to(torch.float32)
            current_batch_loss, _, _ = train_batch(imgs, lbls_true, offset_true, model, criterion, optimizer, scaler, valid = True)
            batch_loss += current_batch_loss
            batch_correct, batch_total = no_of_correct(imgs, lbls_true, model)
            total_correct += batch_correct
            total += batch_total

        epoch_valid_loss = batch_loss / len(validloader)
        epoch_valid_acc = total_correct / total
        valid_losses.append(epoch_valid_loss)
        valid_accuracies.append(epoch_valid_acc)
        
        # Checkpoint
        if epoch_valid_acc >= best_valid_metric:
            print('#### Saving ####')
            save_checkpoint(model, save_path)
            best_valid_metric = epoch_valid_acc
            # If the valid metric has changed set to -1 which will be made to 0, hence resetting the counter for early stopping
            no_change_epoch = -1

        no_change_epoch += 1
        print(f"{'*'*10} EPOCH {epoch:2}/{num_epochs} {'*'*10}")
        print(f'''{"#"*33}
Train Loss: {epoch_train_loss:5.3f}, Train Accuracy: {epoch_train_acc*100:5.2f}
Valid Loss: {epoch_valid_loss:5.3f}, Valid Accuracy: {epoch_valid_acc*100:5.2f}
{"#"*33}''')
        if no_change_epoch >= epoch_threshold:
            break
    return model, train_losses, train_accuracies, valid_losses, valid_accuracies, best_valid_metric

In [ ]:
model, train_losses, train_accuracies, valid_losses, valid_accuracies, best_valid_metric = train(train_loader, valid_loader, rcnn, criterion, optimizer, 
                                                                                                 num_epochs, save_path = save_path)

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(121)
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(train_accuracies, 'r--', label = 'Train')
plt.plot(valid_accuracies, 'g-', label = 'Valid')
plt.legend()
plt.subplot(122)
plt.title('Losses vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(train_losses, 'r--', label = 'Train')
plt.plot(valid_losses, 'g-', label = 'Valid')
plt.legend();

In [ ]:
freeze = False
num_freeze = 0
learning_rate = 1e-05
rcnn = RCNN(base_model=model_name, lmb = lmb, freeze=freeze, num_freeze=num_freeze, dropout_p=dropout_p, idx_to_cat=idx_to_cat)
rcnn = rcnn.to(device)
optimizer = optim.Adam(rcnn.parameters(), lr = learning_rate)
criterion = rcnn.calculate_loss

model, train_losses, train_accuracies, valid_losses, valid_accuracies, best_valid_metric = train(train_loader, valid_loader, rcnn, criterion, optimizer, num_epochs, 
                                                                                                 load_check = True, best_valid_metric = best_valid_metric,
                                                                                                 save_path = save_path)

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(121)
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(train_accuracies, 'r--', label = 'Train')
plt.plot(valid_accuracies, 'g-', label = 'Valid')
plt.legend()
plt.subplot(122)
plt.title('Losses vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(train_losses, 'r--', label = 'Train')
plt.plot(valid_losses, 'g-', label = 'Valid')
plt.legend();